In [155]:
#imports and reading the point cloud

import laspy as las 
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os

pcd = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "pcd.ply"))
num = len(pcd.points)
print("Number of points before downsampling: ", num)
o3d.visualization.draw_geometries([pcd])







Number of points before downsampling:  2016548


In [327]:
#downsample cloud

downpcd = pcd.voxel_down_sample(voxel_size=0.5) #voxel size 0.5 (0.15 also works)
num2 = len(downpcd.points)
print("Number of points after downsampling: ", num2)
print(100-(num2/num * 100),"% less points")
o3d.visualization.draw_geometries([downpcd])

Number of points after downsampling:  367663
-116249.0506329114 % less points


In [328]:
#statistical outlier removal

finalCloud, ind = downpcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2)

num3 = len(finalCloud.points)
print("Number of points removed: ", num2-num3)
print(100-(num3/num2 * 100),"% less points")
print("Number of points left: ", num3) 

o3d.visualization.draw_geometries([finalCloud])

Number of points removed:  7179
1.9526033351193917 % less points
Number of points left:  360484


In [329]:
#clustering

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(finalCloud.cluster_dbscan(eps=1.3, min_points=20, print_progress=True)) #eps = 1.5, min_points = 20 worked best

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

#colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
#colors[labels < 0] = 0
#finalCloud.colors = o3d.utility.Vector3dVector(colors[:, :3]) #Paint point cloud with cluster colors

o3d.visualization.draw_geometries([finalCloud])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 756
point cloud has 756 clusters


In [324]:
o3d.visualization.draw_geometries([finalCloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 


In [279]:
#After the clusters are made, you can use this code to get the individual point clouds into a seperate array
individual_point_clouds = []

for cluster_label in range(max_label + 1):
    
    cluster_indices = np.where(labels == cluster_label)[0]

    cluster_points = finalCloud.select_by_index(cluster_indices)

    individual_point_clouds.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)

In [275]:
#This bit of code takes the first (index 0) point cloud from the array and displays it (If you want)

num = np.random.randint(689)

firstCld = individual_point_clouds[num]

firstCld = o3d.geometry.PointCloud(firstCld) #This actually creates the point cloud from the data in the array
o3d.visualization.draw_geometries([firstCld])

print(firstCld)


PointCloud with 56 points.


In [335]:
#Plane estimation using RANSAC on each element in the list of point clouds (The one that works)

roof_cloud = o3d.geometry.PointCloud() #Creating an empty point cloud to store all the inliers (roofs)
leftover_cloud = o3d.geometry.PointCloud() 
counter = 0


for i in individual_point_clouds:
    num_points = len(np.asarray(i.points))

    if num_points < 500: #Choosing an arbitrary number of points to filter out small clusters
        continue

    counter += 1

    plane_model, inliers = i.segment_plane(distance_threshold=0.1, ransac_n=3, num_iterations=2000)
    roofs.append(inliers)
    [a,b,c,d] = plane_model
    inlier_cloud = i.select_by_index(inliers)
    
    
    inlier_cloud.paint_uniform_color([1.0, 0, 0]) #Paint the inliers red so we can see them easier
    

    #Trying to run RANSAC again on the outlier in the same for loop
    outlier_cloud = i.select_by_index(inliers, invert = True)
    plane_model2, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.1, ransac_n=3, num_iterations=2000) #(distance_threshold=0.1, ransac_n=4, num_iterations=2000)
    [a,b,c,d] = plane_model2
    inlier_cloud2 = outlier_cloud.select_by_index(inliers2)
    inlier_cloud2.paint_uniform_color([0, 0, 1]) #Paint the inliers blue so we can see them easier



    #This part of the code combines the inlier_cloud with the empty roof cloud, creating a big cloud with all inliers
    inlier_cloud_load = np.asarray(inlier_cloud.points) #load points
    inlier_cloud_colors_load = np.asarray(inlier_cloud.colors) #load colors

    new_points = np.concatenate((np.asarray(roof_cloud.points), inlier_cloud_load), axis=0)
    new_colors = np.concatenate((np.asarray(roof_cloud.colors), inlier_cloud_colors_load), axis=0)


    roof_cloud.points = o3d.utility.Vector3dVector(new_points)
    roof_cloud.colors = o3d.utility.Vector3dVector(new_colors)   
    

    #This part of the code creates another cloud for the second ransac

    inlier_cloud_load2 = np.asarray(inlier_cloud2.points) #load points
    inlier_cloud_colors_load2 = np.asarray(inlier_cloud2.colors) #load colors

    new_points2 = np.concatenate((np.asarray(leftover_cloud.points), inlier_cloud_load2), axis=0)
    new_colors2 = np.concatenate((np.asarray(leftover_cloud.colors), inlier_cloud_colors_load2), axis=0)

    leftover_cloud.points = o3d.utility.Vector3dVector(new_points2)
    leftover_cloud.colors = o3d.utility.Vector3dVector(new_colors2)

    

    





In [336]:
o3d.visualization.draw_geometries([leftover_cloud, roof_cloud, finalCloud])
print("Number of clusters with more than 500 points: ", counter)


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] 

In [24]:
#Plane estimation using RANSAC on each element in the list of point clouds

roof_cloud = o3d.geometry.PointCloud() #Creating an empty point cloud to store all the inliers (roofs)
leftover_cloud = o3d.geometry.PointCloud() 
counter = 0


for i in individual_point_clouds:
    num_points = len(np.asarray(i.points))

    if num_points < 1000: #Choosing an arbitrary number of points to filter out small clusters
        continue
    counter += 1

    
    plane_model, inliers = i.segment_plane(distance_threshold=0.1, ransac_n=4, num_iterations=2000)
    [a,b,c,d] = plane_model
    inlier_cloud = i.select_by_index(inliers)
    
    inlier_cloud.paint_uniform_color([1.0, 0, 0]) #Paint the inliers red so we can see them easier
    

    #Trying to run RANSAC again on the outlier in the same for loop
    outlier_cloud = i.select_by_index(inliers, invert = True)
    plane_model2, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.1, ransac_n=4, num_iterations=2000)
    [a,b,c,d] = plane_model2
    inlier_cloud2 = outlier_cloud.select_by_index(inliers2)
    inlier_cloud2.paint_uniform_color([0, 0, 1]) #Paint the inliers blue so we can see them easier



    #This part of the code combines the inlier_cloud with the empty roof cloud, creating a big cloud with all inliers
    inlier_cloud_load = np.asarray(inlier_cloud.points) #load points
    inlier_cloud_colors_load = np.asarray(inlier_cloud.colors) #load colors

    new_points = np.concatenate((np.asarray(roof_cloud.points), inlier_cloud_load), axis=0)
    new_colors = np.concatenate((np.asarray(roof_cloud.colors), inlier_cloud_colors_load), axis=0)


    roof_cloud.points = o3d.utility.Vector3dVector(new_points)
    roof_cloud.colors = o3d.utility.Vector3dVector(new_colors)   
    



    inlier_cloud_load2 = np.asarray(inlier_cloud2.points) #load points
    inlier_cloud_colors_load2 = np.asarray(inlier_cloud2.colors) #load colors

    new_points2 = np.concatenate((np.asarray(leftover_cloud.points), inlier_cloud_load2), axis=0)
    new_colors2 = np.concatenate((np.asarray(leftover_cloud.colors), inlier_cloud_colors_load2), axis=0)

    leftover_cloud.points = o3d.utility.Vector3dVector(new_points2)
    leftover_cloud.colors = o3d.utility.Vector3dVector(new_colors2)

    

    



